<a href="https://colab.research.google.com/github/javvadisurekha/CA_Python/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import openpyxl
import pandas as pd

In [2]:
# working with excel using 'openpyxl' module
excel = openpyxl.Workbook()
sheet = excel.active
sheet.title = "Top 250 Movies"
sheet.append(["Movie Rank", "Movie Name", "Year of Release", "Rating"])


# requesting source website using 'request' module
source = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250')

# Creating BeautifulSoup Object
soup = BeautifulSoup(source.text, 'html.parser')

In [3]:
# Fetching Movies details form soup object
movies_list = soup.find('tbody', class_="lister-list").find_all('tr')
for movie in movies_list:
  name = movie.find('td', class_="titleColumn").a.text
  rank =  movie.find('td', class_="titleColumn").get_text(strip = True).split('.')[0]
  year = movie.find('td', class_="titleColumn").span.text.strip('()')
  rating = movie.find('td', class_="ratingColumn").strong.text

# appending data to excel file 
  sheet.append([rank, name, year, rating])

excel.save('Top 250 Movie.xlsx')

In [4]:
import sqlite3

# import "itertools" to perform forloop on multiple lists at one time 
import itertools

# Creating SQLITE3 Database
db = sqlite3.connect('movies.db')
cur = db.cursor()

# creating dataframe using "pandas"
df = pd.read_excel('Top 250 Movie.xlsx')
df.dropna()

,Movie Rank,Movie Name,Year of Release,Rating
0,1,The Shawshank Redemption,1994,9.2
1,2,The Godfather,1972,9.2
2,3,The Dark Knight,2008,9.0
3,4,The Godfather Part II,1974,9.0
4,5,12 Angry Men,1957,9.0
...,...,...,...,...
245,246,Dersu Uzala,1975,8.0
246,247,Aladdin,1992,8.0
247,248,The Iron Giant,1999,8.0
248,249,The Help,2011,8.0


In [14]:
# max rating
max_rating = df['Rating'].max()
max_id = df['Rating'].idxmax()
max_name = df.loc[max_id]['Movie Name']

In [15]:
# min rating
min_rating = df['Rating'].min()
min_id = df['Rating'].idxmin()
min_name = df.loc[min_id]['Movie Name']

In [16]:
# oldest movie name and year
oldest_movie = df['Year of Release'].min()
old_id = df['Year of Release'].idxmin()
old_name = df.loc[old_id]['Movie Name']

In [17]:
# Newest movie name and year
newest_movie = df['Year of Release'].max()
new_id = df['Year of Release'].idxmax()
new_name = df.loc[new_id]['Movie Name']

In [18]:
#Printing Data

print(f"----------------------------\nMaximum Rating : {max_name} {max_rating}")
print(f"Minimum Rating : {min_name} {min_rating}")
print(f"Oldest Movie : {old_name} {oldest_movie}")
print(f"Newest Movie : {new_name} {newest_movie}\n-----------------------------")

----------------------------
Maximum Rating : The Shawshank Redemption 9.2
Minimum Rating : Ford v Ferrari 8.0
Oldest Movie : The Kid 1921
Newest Movie : Top Gun: Maverick 2022
-----------------------------


In [19]:
# Ninties Movie Data Filtering --------------------------------

movie_old = df['Year of Release'] < 2000

data_dict = df[movie_old].to_dict()

movies_90s = data_dict['Movie Name']
movie_rank = []
movie_name = []
movie_year = []
movie_rating = []

for key, value in data_dict['Movie Rank'].items():
    movie_rank.append(value)

for key, value in data_dict['Movie Name'].items():
    movie_name.append(value)

for key, value in data_dict['Year of Release'].items():
    movie_year.append(value)

for key, value in data_dict['Rating'].items():
    movie_rating.append(value)

# ------------------------------------------

In [20]:
# Table Creation for ninties movies
cur.execute('''CREATE TABLE IF NOT EXISTS `ninties_movies` (id INTEGER PRIMARY KEY, rank INTEGER, title VARCHAR(255), year INTEGER, rating Real)''')

for (a,b,c,d) in zip(movie_rank, movie_name, movie_year, movie_rating):
    # replacing ' from Movie Names
    b = b.replace("'","")

    # CommentIn this if Data Already Inserted before another Run
cur.execute(f"INSERT INTO `ninties_movies` ( `rank`, `title`, `year`, `rating`) VALUES ( {a}, '{b}', {c}, {d})" )

In [21]:
# Twenties Movie Data Filtering --------------------------------

movie_new = df['Year of Release'] >= 2000

data_dict_new = df[movie_new].to_dict()

movies_90s = data_dict['Movie Name']

# Lists to store different column data into individually
movie_rank_new = []
movie_name_new = []
movie_year_new = []
movie_rating_new = []

for key, value in data_dict_new['Movie Rank'].items():
    movie_rank_new.append(value)

for key, value in data_dict_new['Movie Name'].items():
    movie_name_new.append(value)

for key, value in data_dict_new['Year of Release'].items():
    movie_year_new.append(value)

for key, value in data_dict_new['Rating'].items():
    movie_rating_new.append(value)


In [22]:
# Table Creation for twenties movies
cur.execute('''CREATE TABLE IF NOT EXISTS `twenties_movies` (id INTEGER PRIMARY KEY, rank INTEGER, title VARCHAR(255), year INTEGER, rating Real)''')

for (a,b,c,d) in zip(movie_rank_new, movie_name_new, movie_year_new, movie_rating_new):
    # replacing ' from Movie Names
    b = b.replace("'","")

    # CommentIn this if Data Already Inserted before another Run
cur.execute(f"INSERT INTO `twenties_movies` ( `rank`, `title`, `year`, `rating`) VALUES ( {a}, '{b}', {c}, {d})" )
    
db.commit()

In [23]:
cur.execute('''SELECT * FROM twenties_movies''')
results = cur.fetchall()
print(results)

[(1, 249, 'The Help', 2011, 8.0)]


In [24]:
cur.execute('''SELECT * FROM ninties_movies''')
results = cur.fetchall()
print(results)

[(1, 250, 'Gandhi', 1982, 8.0)]
